## https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [99]:
import pandas as pd
import itertools
from tqdm.notebook import tqdm

# https://github.com/UKPLab/sentence-transformers
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")
import calcbench as cb

In [67]:
categories = [
    (
        "environmental",
        """biodiversity, carbon, carbon negative, carbon neutral, carbon zero, clean-up, clean, clean tech, climate change, climate positive, contamination, discharge, emission, energy-efficient, environmental, environmental risk, fuel, fuel efficiency, green, greenhouse gas, hazardous, low carbon, natural resource, net-zero emission, pollution, remediation, sustainability, sustainable, toxic, waste, water, zero carbon, zero net carbon""",
    ),
    (
        "social",
        """accident, antiracism, consumer protection, customer privacy, employee relation, equal, equal pay, equity, gender equality, health, human right, justice, labor relation, labor standard, racial equity, racial awareness, racial justice, working condition""",
    ),
    (
        "goverance",
        """advocacy, antitrust, board independence, code of ethic, compensation, corporate culture, corporate governance, corruption, governance risk, justice, political lobbying, scandal, shareholder right, stability, stewardship, transparency""",
    ),
]

In [57]:
esg_embeddings = model.encode(
    [words for _, words in categories], convert_to_tensor=True
)

In [95]:
search_results = list(
    cb.document_search(
        company_identifiers=cb.tickers(index="DJIA"),
        document_name="ManagementsDiscussionAndAnalysis",
        all_history=True,
        period_type="annual",
    )
)

search_results = [
    s for s in search_results if s.document_type == "10-K"
]  # Remove amended filings

In [100]:
document_contents = [(d, d.get_contents_text()) for d in tqdm(search_results)]

  0%|          | 0/333 [00:00<?, ?it/s]

In [102]:
md_a_embeddings = model.encode(
    [contents for _, contents in document_contents],
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [104]:
cosine_scores = util.pytorch_cos_sim(esg_embeddings, md_a_embeddings)

In [113]:
distances = pd.DataFrame(
    cosine_scores.numpy(),
    columns=pd.MultiIndex.from_tuples([(document.ticker, document.fiscal_year) for document, _ in document_contents], names=['ticker', 'year']),
    index=[c for c, _ in categories],
)

In [114]:
distances.T

environmental    social  goverance
ticker year                                    
MMM    2020       0.077228  0.253213   0.439496
       2019       0.077228  0.253213   0.439496
       2018       0.064323  0.296376   0.440758
       2017       0.064323  0.296376   0.440758
       2016       0.064323  0.296376   0.440758
       2015       0.064323  0.296376   0.440758
       2014       0.064323  0.296376   0.440758
       2013       0.042174  0.304803   0.407609
       2012       0.042174  0.304803   0.407609
       2011       0.042174  0.304803   0.407609
       2010       0.042174  0.304803   0.407609
       2009       0.111995  0.316159   0.431559
AXP    2020       0.072081  0.351238   0.479153
       2019       0.072081  0.351238   0.479153
       2018       0.030910  0.312566   0.446919
       2017       0.068351  0.339271   0.473175
       2016       0.080930  0.334408   0.467387
       2015      -0.048126  0.235690   0.337480
       2014       0.100983  0.376199   0.450088
       2013       0.124777  0.249278   0.360710
       2012       0.140350  0.278982   0.372713
       2011       0.202741  0.284984   0.346475
       2010       0.163985  0.443139   0.573484
       2009       0.142886  0.492625   0.636937
AMGN   2020       0.119456  0.270231   0.435111
       2019       0.119456  0.270231   0.435111
       2018       0.119456  0.270231   0.435111
       2017       0.119456  0.270231   0.435111
       2016       0.119456  0.270231   0.435111
       2015       0.118884  0.274173   0.449034
       2014       0.118884  0.274173   0.449034
       2013       0.075036  0.290406   0.479634
       2012       0.075036  0.290406   0.479634
       2011       0.068371  0.288528   0.480415
       2010       0.068473  0.278673   0.462371
       2009       0.067362  0.275743   0.458800
AAPL   2020       0.130628  0.207154   0.349331
       2019       0.188023  0.270651   0.411581
       2018       0.188023  0.270651   0.411581
       2017       0.188023  0.270651   0.411581
       2016       0.188023  0.270651   0.411581
       2015       0.191258  0.274524   0.417704
       2014       0.191258  0.274524   0.417704
       2013       0.226868  0.266075   0.441959
       2012       0.226868  0.266075   0.441959
       2011       0.111425  0.306066   0.406167
       2010       0.111425  0.306066   0.406167
       2009       0.111425  0.306066   0.406167
BA     2020       0.049635  0.250201   0.326565
       2019       0.049635  0.250201   0.326565
       2018       0.049635  0.250201   0.326565
       2017       0.049635  0.250201   0.326565
       2016       0.064898  0.262650   0.356597
       2015       0.064898  0.262650   0.356597
       2014       0.064898  0.262650   0.356597
       2013       0.064898  0.262650   0.356597
       2012       0.064898  0.262650   0.356597
       2011       0.083011  0.269753   0.347116
       2010       0.083948  0.283511   0.361242
       2009       0.086160  0.276106   0.367098
CAT    2020       0.109100  0.241366   0.337303
       2019       0.088863  0.217350   0.313225
       2018       0.014626  0.277923   0.342397
       2017       0.019776  0.326648   0.365387
       2016       0.108846  0.295772   0.367140
       2015       0.151924  0.264750   0.354599
       2014       0.079701  0.144678   0.244127
       2012       0.034851  0.258500   0.278721
       2011       0.215788  0.397577   0.558484
       2010       0.163074  0.310190   0.312749
       2009       0.080350  0.230139   0.250487
CVX    2019       0.046270  0.302959   0.421872
       2018       0.046270  0.302959   0.421872
       2017       0.083721  0.277963   0.429935
       2016       0.077678  0.396881   0.516209
       2015       0.077678  0.396881   0.516209
       2014       0.068666  0.384611   0.498977
       2013       0.095034  0.382833   0.511194
       2012       0.088496  0.386430   0.506141
       2011       0.211012  0.291303   0.434560
       2010       0.163515  0.294761   0.424532
       2009       0.197030  0

In [82]:
distances.loc[:, ~distances.columns.duplicated()].style.background_gradient(
    cmap="viridis"
)

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,GS,HD,HON,INTC,IBM,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT,DIS
environmental,0.077228,0.072081,0.119456,0.188023,0.049635,0.088863,0.046270,0.208268,0.118117,0.137277,0.044041,0.218505,0.156125,0.112483,0.037996,0.038603,0.110461,0.276271,0.087996,0.109887,0.086627,0.166836,0.138848,0.086009,0.106012,0.094298,0.093081,0.079708,0.051817,0.198753
social,0.253213,0.351238,0.270231,0.270651,0.250201,0.217350,0.302959,0.322666,0.293957,0.246113,0.287698,0.311418,0.188648,0.268946,0.328172,0.294968,0.214728,0.301616,0.170367,0.280293,0.273858,0.306774,0.325204,0.161181,0.328216,0.302468,0.382732,0.337340,0.217662,0.347523
goverance,0.439496,0.479153,0.435111,0.411581,0.326565,0.313225,0.421872,0.447613,0.465742,0.387791,0.412481,0.485787,0.316277,0.302344,0.360157,0.395387,0.300616,0.391904,0.254094,0.415492,0.369767,0.530607,0.450492,0.251981,0.471686,0.348908,0.514012,0.495103,0.381842,0.425977


In [85]:
distances.columns[distances.columns.duplicated()]

Index(['PG'], dtype='object')

In [93]:
[s for s in search_results if s.ticker == "PG"]

[DocumentSearchResults(fact_id=None, entity_name='PROCTER & GAMBLE Co', accession_id=271810, footnote_type=2700, SEC_URL='https://www.sec.gov/Archives/edgar/data/80424/000008042420000059/0000080424-20-000059-index.htm', sec_filing_id=1601824, blob_id='1601824_section270', fiscal_year=2020, fiscal_period='Y', calendar_year=2020, calendar_period='2', filing_date='2020-08-10T00:00:00', received_date='2020-08-10T00:00:00', document_type='10-K/A', guide_link=None, page_url='https://www.sec.gov/Archives/edgar/data/80424/000008042420000059/000008042420000059index.htm/pg-20200630.htm', entity_id=5841, id_detail=False, local_name=None, CIK='0000080424', sec_accession_number='0000080424-20-000059', network_id=None, ticker='PG', filing_type=2, description='PROCTER & GAMBLE Co (PG)', disclosure_type_name='ManagementsDiscussionAndAnalysis', period_end_date='2020-06-30T00:00:00', footnote_type_title='Additional 10-K and 10-Q Sections', content=None),
 DocumentSearchResults(fact_id=None, entity_name=